In [1]:
from helpers.models import Models
from helpers.llm_client import LLMClient
from helpers.functions import *
import pandas as pd
import re
import os
from sklearn.metrics import accuracy_score

# pd.set_option('display.max_rows', None)    # Show all rows
# pd.set_option('display.max_colwidth', None)  # Show full column width

In [1]:
import os

def read_markdown_files_in_date_order(directory):
  """Reads all .md files in a directory and appends their content in date order."""
  files_with_dates = []
  for filename in os.listdir(directory):
    if filename.endswith(".md"):
      # Extract date from filename (assuming format like 'YYYY-MM-DD-title.md')
      date_str = filename.split('-')[0:3]  # Get first 3 parts (YYYY, MM, DD)
      date = '-'.join(date_str)  # Rejoin into 'YYYY-MM-DD'
      files_with_dates.append((date, filename))

  # Sort by date (ascending)
  files_with_dates.sort()

  combined_content = ""
  for date, filename in files_with_dates:
    with open(os.path.join(directory, filename), "r") as f:
      combined_content += f.read() + "\n\n" 
  return combined_content

# Assuming the directory path is correct
files_content = read_markdown_files_in_date_order('/home/jimbo/Desktop/GSoC24/repo/Fossology-gsoc/docs/2024/license-detection/updates')

def write_to_text_file(content, filename="combined_markdown.txt"):
  """Writes the given content to a text file."""
  with open(filename, "w") as f:
    f.write(content)

# Assuming files_content is available from the previous code
write_to_text_file(files_content)

In [15]:
df = pd.read_csv('extras/processedLicenses.csv')
# df[df['shortname'] == 'AAL']
print(df.loc[3, 'text'])

Attribution Assurance License

Copyright (c) 2002 by AUTHOR PROFESSIONAL IDENTIFICATION * URL "PROMOTIONAL SLOGAN FOR AUTHOR'S PROFESSIONAL PRACTICE"

All Rights Reserved

ATTRIBUTION ASSURANCE LICENSE (adapted from the original BSD license)

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the conditions below are met. These conditions require a modest attribution to <AUTHOR> (the "Author"), who hopes that its promotional value may help justify the thousands of dollars in otherwise billable time invested in writing this and other freely available, open-source software.

1. Redistributions of source code, in whole or part and with or without modification (the "Code"), must prominently display this GPG-signed text in verifiable form.

2. Redistributions of the Code in binary form must be accompanied by this GPG-signed text in any documentation and, each time the resulting executable program or a program dependent thereon is lau

In [21]:
# If the keys are not present, add these lines to ~/.bashrc
# sudo gedit ~/.bashrc
# ! Add your own API keys here, currently all 3 give a level of free access with GRoQ being the most free
# export GROQ_API_KEY = ""
# export NVIDIA_API_KEY = ""
# export TOGETHER_API_KEY = ""

In [22]:
# Check to make sure that all API keys are present
os.environ['GROQ_API_KEY'] 
os.environ['NVIDIA_API_KEY']
os.environ['TOGETHER_API_KEY']    
'OK'
#

'OK'

In [23]:
df = pd.read_csv('test-1.csv')
df.head(5)

,Unnamed: 0,index,file path,licenses,file_comments,comments_extracted,response
0,0,0,NomosTestfiles/AAL/AAL.txt,AAL,Attribution Assurance License\nCopyright (c) 2...,False,```\nATTRIBUTION ASSURANCE LICENSE (adapted fr...
1,1,1,NomosTestfiles/AAL/LICENSE,AAL,Copyright (c) 2007 - 2010 by Arslan Hassan et ...,False,```\nATTRIBUTION ASSURANCE LICENSE (adapted fr...
2,2,2,NomosTestfiles/Abstyles/Abstyles.txt,Abstyles,"This is APREAMBL.TEX, version 1.10e, written b...",False,```\nThis program is distributed WITHOUT ANY W...
3,3,3,NomosTestfiles/ACAA/c32001a.ada,Govt-rights\nUnclassifiedLicense,-- C32001A.ADA\n\n-- ...,False,```\n-- Grant of U...
4,4,4,NomosTestfiles/ACE/ACE-copying.html,ACE,<HTML>\n\n<HEAD>\n<TITLE>Copyright and Licensi...,False,"```\nCopyright (c) 1993-2009, all rights reser..."


In [64]:
def parse_response(response):
    try:
        pattern = r"```.*```"
        matches = re.findall(pattern, response, re.DOTALL)
        return matches[0][3:-3]
    except:
        return '' 

df = df[df['response'].notna()]

df['extracted_response'] = df['response'].apply(lambda x: parse_response(x))

print(df.loc[25, 'extracted_response'])


set(CPACK_RPM_PACKAGE_LICENSE "AGPL v.3.0")



In [65]:
def scan(file_comments : list, licenseList : pd.DataFrame): 
    '''
    Scans a file's comments for potential licenses using semantic search and fuzzy string matching.
    '''

    # Append The Fulll License Name, Short License Name, and SPDX-License-Identifier to the license text
    # Some files only contain 'SPDX-License-Identifier: 0BSD' or 'License: 0BSD' and since this agent attempts
    # To match based on license text matching, those will not be identified. Appending those lines
    # To the license text helps this agent identify those cases more often
    def convert(row):
        row['text'] = f"License Name: {row['fullname']} \n License: {row['shortname']} \n SPDX-License-Identifier: {row['shortname']} \n{row['text']}"
        return row
    
    licenseList = licenseList.apply(convert, axis=1)

    # Remove characters not found in license texts
    chars_to_remove = ['—', '…', '•', '§', '«', '»', '„', '・', '−', '*', '>', '<']
    for char_to_remove in chars_to_remove:
        file_comments = file_comments.replace(char_to_remove, '')

    # Separate the comments into single line comments
    file_comments = file_comments.split('\n')

    # Separate each license text line by line and append them to one big list (used for matching)
    # The license_index_map maps each line to the correct license index in the licenseList dataframe
    license_index_map = {}
    all_license_texts = []
    for license_index, license_text in enumerate(licenseList['text']):
        for line in license_text.split('\n'):
            all_license_texts.append(line)
            license_index_map[len(all_license_texts) - 1] = license_index
    
    # Perform first level fuzzy matching for all lines in the comments
    results = []
    fuzzy_similarity_matrix = np.zeros((len(file_comments), len(all_license_texts)))
    for index, comment in enumerate(file_comments):
        for i in range(len(all_license_texts)):
            fuzzy_similarity_matrix[index][i] = fuzz.ratio(comment, all_license_texts[i]) 
        max_score_index = np.argmax(fuzzy_similarity_matrix[index]) 
        results.append(
                    (
                        fuzzy_similarity_matrix[index][max_score_index], 
                        comment
                    )
                ) 
    
    # Try to append lines that match with a similarity score of more than 40% with one of the lines in 
    # Any of the licenses. The goal is to get a bigger and bigger text chunk that matches to a bigger 
    # text chunk in one of the license texts - The bigger the chunk, the more likely that this match is 
    # correct.
    appended_comments = []
    appended_comment = []
    for result in results:
        if result[0] >= 40:
            if appended_comment == [] and result[1] == '':
                continue
            appended_comment.append(result[1])
        else:
            appended_comments.append(appended_comment)
            appended_comment = []

    if len(appended_comment) > 0 and appended_comment not in appended_comments:
        appended_comments.append(appended_comment)
        
    # Attempt the final - second level license match with all the bigger text chunks
    # In some licenses, the license header is used instead of the license text
    # In case that a license has a license header, we match all chunks with both and take 
    # the match with the highest similarity score
    results = []
    for appended_comment in appended_comments:
        appended_comment = "\n".join(appended_comment)
        fuzzy_similarity_matrix_2 = np.zeros(len(licenseList))
        for i in range(len(licenseList)):
            fuzzy_similarity_matrix_2[i] = fuzz.ratio(appended_comment, licenseList.loc[i, 'text'])
            if pd.notna(licenseList.loc[i, 'license_header']):
                license_header_sim_score = fuzz.ratio(appended_comment, licenseList.loc[i, 'license_header'].replace('\n\n', '\n'))
                fuzzy_similarity_matrix_2[i] = license_header_sim_score if license_header_sim_score > fuzzy_similarity_matrix_2[i] else fuzzy_similarity_matrix_2[i]
        top_5_license_text_indices = np.argsort(fuzzy_similarity_matrix_2)[-5:][::-1]
    
        if fuzzy_similarity_matrix_2[top_5_license_text_indices[0]] >= 50:
            results.append({
            "shortname": licenseList.loc[top_5_license_text_indices[0], 'shortname'],
            "sim_score": fuzzy_similarity_matrix_2[top_5_license_text_indices[0]],
            "sim_type": "SemanticSearch-LVD",
            "description": ""
            })
    return results

In [69]:
from tqdm import tqdm

licenseList = pd.read_csv('extras/processedLicenses.csv')

for index, row in tqdm(df.iterrows()):
    results = scan(file_comments=df.loc[index, 'extracted_response'], licenseList=licenseList)
    licenses = []
    for result in results:
        licenses.append(result['shortname'])
    df.loc[index ,'resulting_license'] = '---'.join(licenses)

186it [11:00,  3.55s/it]


In [70]:
def process(row):
    return row['licenses'] in row['resulting_license']
df['accuracy'] = df[['licenses', 'resulting_license']].apply(process, axis=1)
sum(df['accuracy']) / len(df) * 100

38.17204301075269

In [71]:
df['resulting_license']

0                                  AAL
1                                  AAL
2      Linux-man-pages-copyleft-2-para
3                                     
4                                     
                    ...               
194                                   
195                                   
197                            OSL-1.1
198                  SSLeay-standalone
199                               lsof
Name: resulting_license, Length: 186, dtype: object

In [ ]:
test_file_path = 'extras/LastGoodNomosTestfilesScan.txt'
test_file_columns = ['file path', 'licenses']

with open(test_file_path, 'r') as file:
    test_data = file.readlines()

test_df = pd.DataFrame(columns=test_file_columns)

for line in test_data:
    if "contains license(s)" in line:  # Check for the expected pattern
        licenses = line.split("contains license(s) ")[1].strip().split("\\")[0]
        file_path = line.split('File ')[1].split(' contains license(s)')[0].strip()
        licenses = str(licenses)
        licenses = licenses.split(',')
        licenses = '\n'.join(licenses)
        temp_df = pd.DataFrame({'file path': file_path, 'licenses': [licenses]})
        test_df = pd.concat([test_df, temp_df], ignore_index=True)

test_df.loc[1997, 'licenses']

FileNotFoundError: [Errno 2] No such file or directory: 'extras/LastGoodNomosTestfilesScan.txt'

In [6]:
for index, row in test_df.iterrows():
    try:
        with open(os.path.join('extras', row['file path']), "r", encoding='utf-8') as f:
            comments = f.read()
    except:
        print(f'Dropping Index: {index}')
        test_df = test_df.drop(index)
test_df = test_df.reset_index()

NameError: name 'test_df' is not defined

In [6]:
create_license_dataset('extras/license_information/details')
client = LLMClient()

License dataset file created successfully at extras/license_information/license_dataset.csv


In [7]:
def prompt_1(comments : str, comments_extracted: bool):
    return f"""
Task: License Identification in Code Comments or Plain Text

Objective:
Extract the contiguous block (chunk) of text that is most likely to contain license-related information.

Input:
Either:
1. Comment text extracted from a code file (e.g., comments from a .cpp or .py file).
2. The full text of a file that does not typically contain comments (e.g., a NOTICE file).

Guidelines:

* IF COMMENT TEXT IS PROVIDED:
    * License information is typically contained within a single block of comment text.
    * License text often uses keywords like "copyright," "license," "permission," "terms," "distribution," "use," "modification," "attribution," etc.
    * License text might contain specific phrases like "All rights reserved," "MIT License," "Apache License 2.0," etc.

* IF PLAIN TEXT IS PROVIDED:
    * License information is likely to appear as a distinct block within the text.
    * License text is still likely to contain the same keywords and phrases as above.

Instructions:

1. Scan the input text for license-related keywords and phrases.
2. Identify contiguous blocks of text that have a higher density of these keywords.
3. Among these blocks, select all that:
    * Contain the most specific license phrases (e.g., "MIT License," "Apache License 2.0").
    * Appear most similar to typical license language (e.g., legalistic tone, reference to rights and permissions).
    * If applicable, align with any known conventions for licenses within the file type (e.g., header comments in code files).
4. If no block meets the criteria, return "No license information found."

Additional Considerations:

* Be mindful of false positives (e.g., discussions about licenses, legal notices, etc., that are not the actual license text).
* Consider the length of the block. License text can vary in length, but extremely short or overly long blocks might be less likely.
* If the input is from a code file's comments, prioritize blocks near the beginning of the file, as licenses are often placed there.

Output:

A single contiguous block of text representing the most likely license information, or "No license information found."

Output Format:
    Findings:
    [Extracted License Relevant Chunk] or [No license information found.]

{'File Comments:' if comments_extracted else 'File Text:'}
{comments}
"""

In [8]:
def prompt_2(comments : str, comments_extracted: bool):
    return f"""
Task: License Identification in Code Comments or Plain Text

Objective:
Extract the contiguous block (chunk) of text that is most likely to contain license-related information.

Input:
    Either:
    1. Comment text extracted from a code file (e.g., comments from a .cpp or .py file).
    2. The full text of a file that does not typically contain comments (e.g., a NOTICE file).

Guidelines:

* IF COMMENT TEXT IS PROVIDED:
    * License information is typically contained within a single block of comment text.
    * License text often uses keywords like "copyright," "license," "permission," "terms," "distribution," "use," "modification," "attribution," etc.
    * License text might contain specific phrases like "All rights reserved," "MIT License," "Apache License 2.0," etc.

* IF PLAIN TEXT IS PROVIDED:
    * License information is likely to appear as a distinct block within the text.
    * License text is still likely to contain the same keywords and phrases as above.

Instructions:

1. Scan the input text for license-related keywords and phrases.
2. Identify contiguous blocks of text that have a higher density of these keywords.
3. Among these blocks, select the one that:
    * Contains the most specific license phrases (e.g., "MIT License," "Apache License 2.0").
    * Appears most similar to typical license language (e.g., legalistic tone, reference to rights and permissions).
    * If applicable, aligns with any known conventions for licenses within the file type (e.g., header comments in code files).
4. If no block meets the criteria, enclose the text "No license information found." in triple backticks (```).
5. Enclose the selected block of text (or the "No license information found." message) within triple backticks (```) and return this as the output.
6. **Crucially, preserve all original spacing, indentation, and line breaks within the selected block of text.**

Additional Considerations:

* Be mindful of false positives (e.g., discussions about licenses, legal notices, etc., that are not the actual license text).
* Consider the length of the block. License text can vary in length, but extremely short or overly long blocks might be less likely.
* If the input is from a code file's comments, prioritize blocks near the beginning of the file, as licenses are often placed there.

Output:

A single contiguous block of text representing the most likely license information, or "No license information found."

{'File Comments:' if comments_extracted else 'File Text:'}
{comments}
"""

In [9]:
def prompt_3(comments: str, comments_extracted: bool):
    return f"""
Task: License Identification in Code Comments or Plain Text

Objective:
Extract the section of text that is most likely to contain license-related information. This section can be a single block of text or multiple paragraphs separated by spaces, lines, or whitespaces.

Input:
    Either:
    1. Comment text extracted from a code file (e.g., comments from a .cpp or .py file).
    2. The full text of a file that does not typically contain comments (e.g., a NOTICE file).

Guidelines:

* IF COMMENT TEXT IS PROVIDED:
    * License information may be spread across multiple comment blocks or paragraphs.
    * Focus on contiguous sections with a higher density of license keywords.
    * Look for common license phrases like "All rights reserved," "MIT License," "Apache License 2.0," etc.

* IF PLAIN TEXT IS PROVIDED:
    * License information might be present as a distinct section with potential separations.
    * Consider both keywords and phrases, as well as overall structure and context.

Instructions:

1. Scan the input text for license-related keywords and phrases.
2. Identify sections of text (single or multiple paragraphs) with a higher density of these keywords.
3. Among these sections, select the one that:
    * Contains the most specific license phrases.
    * Appears most similar to typical license language (legalistic tone, rights, permissions).
    * Aligns with conventions for licenses within the file type (if applicable).
4. If no section meets the criteria, return "No license information found."
5. Enclose the selected section (or the "No license information found." message) within triple backticks (```) and return this as the output.
6. **Crucially, preserve all original spacing, indentation, and line breaks within the selected section.**

Additional Considerations:

* Be mindful of false positives (discussions about licenses that are not the actual text).
* If the input is from code file comments, prioritize sections near the beginning.

Output:

The most likely license information as one or more text sections, or "No license information found."

{'File Comments:' if comments_extracted else 'File Text:'}
{comments}
"""


In [10]:
def prompt_4(comments : str, comments_extracted: bool):
    return f"""
Task: License Identification in Code Comments or Plain Text

Objective:
Extract the contiguous block (chunk) of text that is most likely to contain license-related information.

Input:
    Either:
    1. Comment text extracted from a code file (e.g., comments from a .cpp or .py file).
    2. The full text of a file that does not typically contain comments (e.g., a NOTICE file).

Guidelines:

* IF COMMENT TEXT IS PROVIDED:
    * License information is typically contained within a single block of comment text.
    * License text often uses keywords like "copyright," "license," "permission," "terms," "distribution," "use," "modification," "attribution," etc.
    * License text might contain specific phrases like "All rights reserved," "MIT License," "Apache License 2.0," etc.

* IF PLAIN TEXT IS PROVIDED:
    * License information is likely to appear as a distinct block within the text.
    * License text is still likely to contain the same keywords and phrases as above.

Instructions:

1. Scan the input text for license-related keywords and phrases.
2. Identify contiguous blocks of text that have a higher density of these keywords.
3. Among these blocks, select the one that:
    * Contains the most specific license phrases (e.g., "MIT License," "Apache License 2.0").
    * Appears most similar to typical license language (e.g., legalistic tone, reference to rights and permissions).
    * If applicable, aligns with any known conventions for licenses within the file type (e.g., header comments in code files).
4. If no block meets the criteria, enclose the text "No license information found." in triple backticks (```).
5. **Crucially, preserve all original spacing, indentation, and line breaks within the selected block of text.** This includes:
    * Empty lines between paragraphs
    * Leading spaces or tabs for indentation
    * Newline characters (\n) at the end of each line
6. Enclose the selected block of text (or the "No license information found." message) within triple backticks (```) and return this as the output. 

Additional Considerations:

* Be mindful of false positives (e.g., discussions about licenses, legal notices, etc., that are not the actual license text).
* Consider the length of the block. License text can vary in length, but extremely short or overly long blocks might be less likely.
* If the input is from a code file's comments, prioritize blocks near the beginning of the file, as licenses are often placed there.

Output:

A single contiguous block of text representing the most likely license information, or "No license information found."

{'File Comments:' if comments_extracted else 'File Text:'}
{comments}
"""

In [11]:
test_df.head(1)

,index,file path,licenses
0,0,NomosTestfiles/AAL/AAL.txt,AAL


In [11]:
test = extract_comments(test_df)
print(test.loc[4, 'file_comments'])

<HTML>

<HEAD>
<TITLE>Copyright and Licensing Information for ACE, TAO, CIAO, DAnCE, and CoSMIC</TITLE>

<BODY text = "#000000"
link="#000fff"
vlink="#ff0f0f"
bgcolor="#ffffff">

<HR>

<H3>Copyright and Licensing Information for ACE<sup><font
size=-2>(TM)</font></sup>, TAO<sup><font
size=-2>(TM)</font></sup>, CIAO<sup><font
size=-2>(TM)</font></sup>, DAnCE<sup><font
size=-2>(TM)</font></sup>, and
CoSMIC<sup><font
size=-2>(TM)</font></sup></H3>

<A HREF="http://www.cs.wustl.edu/~schmidt/ACE.html">ACE</a><sup><font
size=-2>(TM)</font></sup>, <A
HREF="http://www.cs.wustl.edu/~schmidt/TAO.html">TAO</A><sup><font
size=-2>(TM)</font></sup>, <A
HREF="http://www.dre.vanderbilt.edu/CIAO/">CIAO</A><sup><font
size=-2>(TM)</font></sup>, DAnCE><sup><font size=-2>(TM)</font></sup>,
and
 <A HREF="http://www.dre.vanderbilt.edu/cosmic/">CoSMIC</A><sup><font
size=-2>(TM)</font></sup> (henceforth referred to as "DOC software")
are copyrighted by <A
HREF="http://www.dre.vanderbilt.edu/~schmidt/">Douglas C

In [12]:
results_1 = client.process_dataset_license_relevant_chunks(test_df.loc[0:1], Models.LLAMA_3_8b, prompt_1,
                                                         'test-1')

/home/jimbo/Desktop/GSoC24/repo/GSoC24/helpers/llm_client.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'response'] = self._infer(model, prompt, temperature)


In [13]:
results_2 = client.process_dataset_license_relevant_chunks(test_df.loc[0:1], Models.GEMMA_2_9b, prompt_2,
                                                         'test-1')

/home/jimbo/Desktop/GSoC24/repo/GSoC24/helpers/llm_client.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'response'] = self._infer(model, prompt, temperature)


In [14]:
results_3 = client.process_dataset_license_relevant_chunks(test_df.loc[0:1], Models.LLAMA_3_1_8b, prompt_3,
                                                         'test-1')

/home/jimbo/Desktop/GSoC24/repo/GSoC24/helpers/llm_client.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'response'] = self._infer(model, prompt, temperature)


In [13]:
results_4 = client.process_dataset_license_relevant_chunks(test_df.loc[0:199], Models.TOGETHER_GEMMA_2_9b, prompt_4,
                                                         'test-1', log_every=1, retry_fails=False)

2024-08-14 11:32:20.733 | INFO     | helpers.llm_client:process_dataset_license_relevant_chunks:374 - Processing index: 0
/home/jimbo/Desktop/GSoC24/repo/GSoC24/helpers/llm_client.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, 'response'] = self._infer(model, prompt, temperature)
2024-08-14 11:32:26.758 | INFO     | helpers.llm_client:process_dataset_license_relevant_chunks:374 - Processing index: 1
2024-08-14 11:32:32.602 | INFO     | helpers.llm_client:process_dataset_license_relevant_chunks:374 - Processing index: 2
2024-08-14 11:32:34.177 | INFO     | helpers.llm_client:process_dataset_license_relevant_chunks:374 - Processing index: 3
2024-08-14 11:32:38.289 | INFO     | helpers.llm_client:process_dataset_license_relevant

In [14]:
print(results_4.loc[0, 'response'])

```
ATTRIBUTION ASSURANCE LICENSE (adapted from the original BSD license)
Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the conditions below are met.
These conditions require a modest attribution to <AUTHOR> (the
"Author"), who hopes that its promotional value may help justify the
thousands of dollars in otherwise billable time invested in writing
this and other freely available, open-source software.

1. Redistributions of source code, in whole or part and with or without
modification (the "Code"), must prominently display this GPG-signed
text in verifiable form.
2. Redistributions of the Code in binary form must be accompanied by
this GPG-signed text in any documentation and, each time the resulting
executable program or a program dependent thereon is launched, a
prominent display (e.g., splash screen or banner text) of the Author's
attribution information, which includes:
(a) Name ("AUTHOR"),
(b) Professional identificat

In [38]:
print(results_4.loc[4, 'file_comments'])

<HTML>

<HEAD>
<TITLE>Copyright and Licensing Information for ACE, TAO, CIAO, DAnCE, and CoSMIC</TITLE>

<BODY text = "#000000"
link="#000fff"
vlink="#ff0f0f"
bgcolor="#ffffff">

<HR>

<H3>Copyright and Licensing Information for ACE<sup><font
size=-2>(TM)</font></sup>, TAO<sup><font
size=-2>(TM)</font></sup>, CIAO<sup><font
size=-2>(TM)</font></sup>, DAnCE<sup><font
size=-2>(TM)</font></sup>, and
CoSMIC<sup><font
size=-2>(TM)</font></sup></H3>

<A HREF="http://www.cs.wustl.edu/~schmidt/ACE.html">ACE</a><sup><font
size=-2>(TM)</font></sup>, <A
HREF="http://www.cs.wustl.edu/~schmidt/TAO.html">TAO</A><sup><font
size=-2>(TM)</font></sup>, <A
HREF="http://www.dre.vanderbilt.edu/CIAO/">CIAO</A><sup><font
size=-2>(TM)</font></sup>, DAnCE><sup><font size=-2>(TM)</font></sup>,
and
 <A HREF="http://www.dre.vanderbilt.edu/cosmic/">CoSMIC</A><sup><font
size=-2>(TM)</font></sup> (henceforth referred to as "DOC software")
are copyrighted by <A
HREF="http://www.dre.vanderbilt.edu/~schmidt/">Douglas C

In [32]:
results_4_ = results_4[results_4['response'].notna()]

In [34]:
import difflib

def is_similar_substring(substring, mainstring, threshold=0.9):
    matcher = difflib.SequenceMatcher(None, substring, mainstring)
    matching_blocks = matcher.get_matching_blocks()
    matching_text_length = sum(block.size for block in matching_blocks)
    similarity_ratio = matching_text_length / len(substring)  
    return similarity_ratio >= threshold


for index, row in results_4_.iterrows():
    file_comments = row['file_comments']
    response = row['response']
    code_blocks = re.findall(r"```.*```", response, re.DOTALL)  
    for block in code_blocks:
        if not is_similar_substring(block, file_comments, threshold=0.9):
            results_4_.loc[index, 'license_relevant_chunk_found'] = False
            print(f"No similar code block found in file_comments for index {index}:")
        else:
            results_4_.loc[index, 'license_relevant_chunk_found'] = True
results_4_['license_relevant_chunk_found'] = results_4_['license_relevant_chunk_found'].apply(lambda x: x if x in [True, False] else True)
print('Extraction Accuracy: ', accuracy_score([True] * len(results_4_), results_4_['license_relevant_chunk_found'].to_list()))

No similar code block found in file_comments for index 23:
No similar code block found in file_comments for index 25:
No similar code block found in file_comments for index 29:
No similar code block found in file_comments for index 31:
No similar code block found in file_comments for index 32:
No similar code block found in file_comments for index 55:
No similar code block found in file_comments for index 56:
No similar code block found in file_comments for index 65:
No similar code block found in file_comments for index 73:
No similar code block found in file_comments for index 106:
No similar code block found in file_comments for index 120:
No similar code block found in file_comments for index 122:
No similar code block found in file_comments for index 124:
No similar code block found in file_comments for index 139:
No similar code block found in file_comments for index 141:
No similar code block found in file_comments for index 165:
No similar code block found in file_comments for 

/tmp/ipykernel_46958/3462747271.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_4_['license_relevant_chunk_found'] = results_4_['license_relevant_chunk_found'].apply(lambda x: x if x in [True, False] else True)


In [14]:
results_df = pd.read_csv('results/test-1.csv') # in spreadsheet - 2 (2 -> 0, etc.)

In [21]:
print(results_df.loc[27, 'response'])

nan


In [45]:
print(results_4_.loc[19, 'response'])

```
Academic Free License ("AFL") v. 3.0

This Academic Free License (the "License") applies to any original work
of authorship (the "Original Work") whose owner (the "Licensor") has
placed the following licensing notice adjacent to the copyright notice
for the Original Work:

Licensed under the Academic Free License version 3.0

1) Grant of Copyright License. Licensor grants You a worldwide,
royalty-free, non-exclusive, sublicensable license, for the duration of
the copyright, to do the following:

	a) to reproduce the Original Work in copies, either alone or as
	   part of a collective work;

	b) to translate, adapt, alter, transform, modify, or arrange the
	   Original Work, thereby creating derivative works ("Derivative
	   Works") based upon the Original Work;

	c) to distribute or communicate copies of the Original Work and
	   Derivative Works to the public, UNDER ANY LICENSE OF YOUR
	   CHOICE THAT DOES NOT CONTRADICT THE TERMS AND CONDITIONS,
	   INCLUDING LICENSOR'S RESERVED 

In [46]:
print(results_4_.loc[19, 'file path'], results_4_.loc[19, 'licenses'])

NomosTestfiles/AFL/LICENSE AFL-3.0


In [47]:
print(results_4.loc[19, 'file_comments'])

Most of MASON is licensed under the Academic Free License, version 3.0,
with the exception of the following files:

	PngEncoder.java			Artistic License
	MovieEncoder.java		[Partial] Sun Open Source License
	WireFrameBoxPortrayal3D.java	[Partial] Sun Open Source License
	ToolTipBehavior.java		[Partial] Sun Open Source License
	SelectionBehavior.java		[Partial] Sun Open Source License
	GullCG.java			Sun Open Source License
	MersenneTwisterFast.java	BSD License

For all remaining files, the Academic Free License is specified below.


Academic Free License ("AFL") v. 3.0

This Academic Free License (the "License") applies to any original work
of authorship (the "Original Work") whose owner (the "Licensor") has
placed the following licensing notice adjacent to the copyright notice
for the Original Work:

Licensed under the Academic Free License version 3.0

1) Grant of Copyright License. Licensor grants You a worldwide,
royalty-free, non-exclusive, sublicensable license, for the duration of

In [4]:
meh = pd.read_csv('processedLicenses.csv')

In [5]:
meh.columns

Index(['shortname', 'fullname', 'text', 'license_header', 'url', 'deprecated',
       'osi_approved', 'isException', 'processed_fullname', 'processed_header',
       'processed_text'],
      dtype='object')

In [6]:
meh[meh['shortname'] == 'LAL-1.2']

,shortname,fullname,text,license_header,url,deprecated,osi_approved,isException,processed_fullname,processed_header,processed_text
402,LAL-1.2,Licence Art Libre 1.2,Licence Art Libre\n[ Copyleft Attitude ]\n\nVe...,NaN,http://artlibre.org/licence/lal/licence-art-li...,False,False,False,licence art libre 1 2,NaN,licence art libre copyleft attitude version 1 ...


In [7]:
print(meh.loc[402, 'text'] )

Licence Art Libre
[ Copyleft Attitude ]

Version 1.2

Préambule :

Avec cette Licence Art Libre, l’autorisation est donnée de copier, de diffuser et de transformer librement les oeuvres dans le respect des droits de l’auteur.

Loin d’ignorer les droits de l’auteur, cette licence les reconnaît et les protège. Elle en reformule le principe en permettant au public de faire un usage créatif des oeuvres d’art.
Alors que l’usage fait du droit de la propriété littéraire et artistique conduit à restreindre l’accès du public à l’oeuvre, la Licence Art Libre a pour but de le favoriser.
L’intention est d’ouvrir l’accès et d’autoriser l’utilisation des ressources d’une oeuvre par le plus grand nombre. En avoir jouissance pour en multiplier les réjouissances, créer de nouvelles conditions de création pour amplifier les possibilités de création. Dans le respect des auteurs avec la reconnaissance et la défense de leur droit moral.

En effet, avec la venue du numérique, l’invention de l’internet et de

In [25]:
print(meh.loc[45, 'license_header'] )

Copyright [yyyy] [name of copyright owner]

Licensed under the Apache License, Version 2.0 (the "License");

you may not use this file except in compliance with the License.

You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software

distributed under the License is distributed on an "AS IS" BASIS,

WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and

limitations under the License.




In [31]:
meh1 = pd.read_csv('extras/license_information/license_dataset.csv')
meh1.loc[0, 'License Text']

'\nSPDX-License-Identifier: CNRI-Jython\n\nLicense Name: CNRI Jython License\n\n\n1. This LICENSE AGREEMENT is between the Corporation for National Research Initiatives, having an office at 1895 Preston White Drive, Reston, VA 20191 ("CNRI"), and the Individual or Organization ("Licensee") accessing and using JPython version 1.1.x in source or binary form and its associated documentation as provided herein ("Software").\n\n2.  Subject to the terms and conditions of this License Agreement, CNRI hereby grants Licensee a non-exclusive, non-transferable, royalty-free, world-wide license to reproduce, analyze, test, perform and/or display publicly, prepare derivative works, distribute, and otherwise use the Software alone or in any derivative version, provided, however, that CNRI\'s License Agreement and CNRI\'s notice of copyright, i.e., “Copyright (c) 1996-1999 Corporation for National Research Initiatives; All Rights Reserved” are both retained in the Software, alone or in any derivative

In [32]:
license_index_map = {}
all_license_texts = []
for license_index, license_text in enumerate(meh1['License Text']):
    for line in license_text.split('\n'):
        all_license_texts.append(line)
        license_index_map[len(all_license_texts) - 1] = license_index

In [34]:
print(all_license_texts[0:10])

['', 'SPDX-License-Identifier: CNRI-Jython', '', 'License Name: CNRI Jython License', '', '', '1. This LICENSE AGREEMENT is between the Corporation for National Research Initiatives, having an office at 1895 Preston White Drive, Reston, VA 20191 ("CNRI"), and the Individual or Organization ("Licensee") accessing and using JPython version 1.1.x in source or binary form and its associated documentation as provided herein ("Software").', '', "2.  Subject to the terms and conditions of this License Agreement, CNRI hereby grants Licensee a non-exclusive, non-transferable, royalty-free, world-wide license to reproduce, analyze, test, perform and/or display publicly, prepare derivative works, distribute, and otherwise use the Software alone or in any derivative version, provided, however, that CNRI's License Agreement and CNRI's notice of copyright, i.e., “Copyright (c) 1996-1999 Corporation for National Research Initiatives; All Rights Reserved” are both retained in the Software, alone or in